In [1]:
stock = 'QCOM'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    58.081139
2018-07-23    58.546898
2018-07-24    58.318970
2018-07-25    58.883827
2018-07-26    63.006290
2018-07-27    62.124321
2018-07-30    61.480186
2018-07-31    63.511684
2018-08-01    63.769341
2018-08-02    64.185547
2018-08-03    64.809868
2018-08-06    65.136894
2018-08-07    64.849510
2018-08-08    64.572037
2018-08-09    64.492760
2018-08-10    64.235100
2018-08-13    64.254921
2018-08-14    64.839600
2018-08-15    65.087341
2018-08-16    65.166618
2018-08-17    65.463913
2018-08-20    65.563011
2018-08-21    65.949493
2018-08-22    66.236870
2018-08-23    66.256691
2018-08-24    66.544075
2018-08-27    66.732361
2018-08-28    69.150345
2018-08-29    68.575577
2018-08-30    67.525146
                ...    
2018-09-17    73.239998
2018-09-18    74.720001
2018-09-19    74.059998
2018-09-20    74.599998
2018-09-21    73.699997
2018-09-24    73.580002
2018-09-25    72.739998
2018-09-26    71.760002
2018-09-27    71.709999
2018-09-28    72.029999
2018-10-01 

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


60.45909986375617
[59.20769118 58.72750533 59.38078288 60.78349285 64.19602708]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")